In [ ]:
import cv2
import numpy as np
import os
import math # For angle calculations
import joblib # For loading models and scalers
import pandas as pd # Used to define feature_columns consistently

# --- Configuration: Model & Scaler Paths ---
# IMPORTANT: These paths must point to where your trained model and scaler are saved.
MODEL_DIR = "trained_model" # Directory created by train_svr_model.py
SVR_MODEL_FILENAME = "svr_deviation_model.joblib"
SCALER_FILENAME = "feature_scaler.joblib"

# --- Configuration: Unseen Data Folder ---
# IMPORTANT: Adjust this path to the folder containing your unseen images.
UNSEEN_DATA_FOLDER = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data"

# --- Configuration: Hardcoded Parameters (MUST MATCH THOSE USED FOR FEATURE EXTRACTION & TRAINING) ---
# Copy these values EXACTLY from your 'feature_extractor.py' or 'train_svr_model.py' script.

# L*a*b* Color Space Ranges for Orange Line
# L*a*b* Color Space Ranges for Orange Line
LOWER_L_TUNED = 147
UPPER_L_TUNED = 221
LOWER_A_TUNED = 135
UPPER_A_TUNED = 157
LOWER_B_TUNED = 135
UPPER_B_TUNED = 161

# Morphological Kernel Size for Color Mask
COLOR_MORPH_KERNEL_SIZE_TUNED = 3

# Edge Pre-processing Filter (Bilateral or Gaussian)
USE_BILATERAL_FILTER_TUNED = 1 # 0 for Gaussian, 1 for Bilateral
BILATERAL_D_TUNED = 6
BILATERAL_SIGMA_COLOR_TUNED = 7
BILATERAL_SIGMA_SPACE_TUNED = 200

# Canny Edge Detector Thresholds
CANNY_THRESH1_TUNED = 3
CANNY_THRESH2_TUNED = 93

# Morphological Kernel Size for Edge Mask
EDGE_MORPH_KERNEL_SIZE_TUNED = 25

# Hough Line Transform Parameters
HOUGH_THRESHOLD_TUNED =15
HOUGH_MIN_LENGTH_TUNED =160
HOUGH_MAX_GAP_TUNED = 45

# Line Filtering Parameters (based on angle)
MAX_LINE_ANGLE_DEG_TUNED = 88 # Maximum allowed deviation from horizontal (0 or 180 degrees)

# Image Cropping Parameter
CROP_PERCENT_FROM_TOP_TUNED = 57 # Percentage of image height to crop from the top

# --- Helper Function to Load Model and Scaler ---
def load_model_and_scaler(model_dir, model_name, scaler_name):
    """Loads the pre-trained SVR model and StandardScaler."""
    try:
        model_path = os.path.join(model_dir, model_name)
        scaler_path = os.path.join(model_dir, scaler_name)
        
        loaded_model = joblib.load(model_path)
        loaded_scaler = joblib.load(scaler_path)
        print(f"✅ Successfully loaded model from: {model_path}")
        print(f"✅ Successfully loaded scaler from: {scaler_path}")
        return loaded_model, loaded_scaler
    except FileNotFoundError as e:
        print(f"ERROR: Model or scaler file not found. Ensure '{model_dir}' exists and contains '{model_name}' and '{scaler_name}'.")
        print(f"Details: {e}")
        exit()
    except Exception as e:
        print(f"ERROR: An error occurred while loading the model or scaler: {e}")
        exit()

# --- Feature Columns (MUST BE IN THE EXACT SAME ORDER AS USED FOR TRAINING) ---
# This list must precisely match the 'feature_columns' in your train_svr_model.py script.
FEATURE_COLUMNS = [
    'is_line_detected',
    'centroid_x_cropped',
    'centroid_y_cropped',
    'num_line_segments',
    'total_line_pixel_length',
    'avg_line_angle_deg',
    'std_line_angle_deg',
    'min_line_y_cropped',
    'max_line_y_cropped',
    'fitted_line_slope_deg',
    'fitted_line_x_intercept_at_bottom_cropped',
    'fitted_line_x_intercept_at_top_cropped'
]


# --- Function to Extract Features from a Single Image (Identical to training logic) ---
def extract_features_for_prediction(image, params):
    """
    Processes a single image to detect the orange lane line and extract relevant features.
    This function is identical to the one used during feature extraction for training.
    """
    
    # Initialize all features with default values
    features_dict = {
        "is_line_detected": 0,
        "centroid_x_cropped": -1.0,
        "centroid_y_cropped": -1.0,
        "num_line_segments": 0,
        "total_line_pixel_length": 0.0,
        "avg_line_angle_deg": 0.0,
        "std_line_angle_deg": 0.0,
        "min_line_y_cropped": -1.0,
        "max_line_y_cropped": -1.0,
        "fitted_line_slope_deg": 0.0,
        "fitted_line_x_intercept_at_bottom_cropped": -1.0,
        "fitted_line_x_intercept_at_top_cropped": -1.0,
    }

    # Store a copy of the original image for drawing annotations later
    annotated_image = image.copy()
    original_h, original_w, _ = image.shape
    center_x_full_image = original_w // 2

    # --- 1. Crop from top ---
    crop_y = int(original_h * params["crop_percent_from_top"] / 100)
    if crop_y >= original_h:
        crop_y = original_h - 1 # Ensure crop_y is within bounds
    cropped_image = image[crop_y:, :].copy()
    cropped_h, cropped_w, _ = cropped_image.shape

    # --- 2. Color Masking (L*a*b*) ---
    lab_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_cropped)
    
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l_channel)
    lab_eq = cv2.merge([l_eq, a_channel, b_channel])
    
    blurred_lab_eq = cv2.medianBlur(lab_eq, 5) 

    lower_orange_lab = np.array([params["lower_l"], params["lower_a"], params["lower_b"]])
    upper_orange_lab = np.array([params["upper_l"], params["upper_a"], params["upper_b"]])
    color_mask = cv2.inRange(blurred_lab_eq, lower_orange_lab, upper_orange_lab)

    # --- 3. Morphological Operations on Color Mask ---
    color_morph_kernel = np.ones((params["color_morph_kernel_size"], params["color_morph_kernel_size"]), np.uint8)
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1) 
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)
    
    # --- 4. Edge Detection (Canny) ---
    gray_cropped = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    
    if params["use_bilateral_filter"]:
        filtered_gray = cv2.bilateralFilter(gray_cropped, params["bilateral_d"], 
                                             params["bilateral_sigma_color"], params["bilateral_sigma_space"])
    else:
        filtered_gray = cv2.GaussianBlur(gray_cropped, (5, 5), 0)

    edge_mask = cv2.Canny(filtered_gray, params["canny_thresh1"], params["canny_thresh2"])

    edge_morph_kernel = np.ones((params["edge_morph_kernel_size"], params["edge_morph_kernel_size"]), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)

    # --- 5. Combine Color Mask and Morphed Edge Mask ---
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    # --- 6. Line Detection with Hough Transform ---
    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180, 
                            params["hough_threshold"], 
                            minLineLength=params["hough_min_length"], 
                            maxLineGap=params["hough_max_gap"])

    # --- 7. Filter Detected Lines and Calculate Features ---
    filtered_lines = []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle_rad = math.atan2(y2 - y1, x2 - x1)
            angle_deg = math.degrees(angle_rad)

            if abs(angle_deg) < params["max_line_angle_deg"] or abs(angle_deg - 180) < params["max_line_angle_deg"]:
                filtered_lines.append(line[0]) # Store as flat list [x1, y1, x2, y2]
                cv2.line(annotated_image, (x1, y1 + crop_y), (x2, y2 + crop_y), (0, 255, 0), 2) # Green lines

    if filtered_lines:
        features_dict["is_line_detected"] = 1
        all_x_coords = []
        all_y_coords = []
        all_line_angles = []
        all_line_lengths = []

        for line_seg in filtered_lines:
            x1, y1, x2, y2 = line_seg
            all_x_coords.extend([x1, x2])
            all_y_coords.extend([y1, y2])

            angle_rad = math.atan2(y2 - y1, x2 - x1)
            all_line_angles.append(math.degrees(angle_rad))
            
            length = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            all_line_lengths.append(length)
        
        features_dict["centroid_x_cropped"] = np.mean(all_x_coords)
        features_dict["centroid_y_cropped"] = np.mean(all_y_coords)

        # Deviation Score is calculated here but not used for model input, just for visual check
        # It's our target variable, so we don't return it as a feature for prediction
        # If the centroid_x_cropped is needed for some other purpose, keep it.
        # deviation_score = (features_dict["centroid_x_cropped"] - center_x_full_image) / center_x_full_image
        # features_dict["deviation_score"] = deviation_score # This would be ground truth, not a feature

        features_dict["num_line_segments"] = len(filtered_lines)
        features_dict["total_line_pixel_length"] = np.sum(all_line_lengths)

        if all_line_angles:
            features_dict["avg_line_angle_deg"] = np.mean(all_line_angles)
            if len(all_line_angles) > 1:
                features_dict["std_line_angle_deg"] = np.std(all_line_angles)
            else:
                features_dict["std_line_angle_deg"] = 0.0

        if all_y_coords:
            features_dict["min_line_y_cropped"] = np.min(all_y_coords)
            features_dict["max_line_y_cropped"] = np.max(all_y_coords)

        if len(all_x_coords) > 1 and len(np.unique(all_x_coords)) > 1:
            try:
                slope, intercept = np.polyfit(all_x_coords, all_y_coords, 1)
                features_dict["fitted_line_slope_deg"] = math.degrees(math.atan(slope))

                if slope != 0:
                    features_dict["fitted_line_x_intercept_at_bottom_cropped"] = (cropped_h - 1 - intercept) / slope
                    features_dict["fitted_line_x_intercept_at_top_cropped"] = (0 - intercept) / slope
                else: # Horizontal line
                    features_dict["fitted_line_x_intercept_at_bottom_cropped"] = features_dict["centroid_x_cropped"]
                    features_dict["fitted_line_x_intercept_at_top_cropped"] = features_dict["centroid_x_cropped"]
            except np.linalg.LinAlgError:
                # Vertical line case
                features_dict["fitted_line_slope_deg"] = 90.0 # or -90 depending on orientation
                features_dict["fitted_line_x_intercept_at_bottom_cropped"] = features_dict["centroid_x_cropped"]
                features_dict["fitted_line_x_intercept_at_top_cropped"] = features_dict["centroid_x_cropped"]
            except Exception as e:
                print(f"Warning: Error fitting line during prediction feature extraction: {e}")

        # Draw centroid on annotated image
        cx_img = int(features_dict["centroid_x_cropped"])
        # Choose a representative y for the centroid for drawing (e.g., middle of the crop region)
        cy_img = int(features_dict["centroid_y_cropped"] + crop_y) 
        cv2.circle(annotated_image, (cx_img, cy_img), 6, (0, 0, 255), -1) # Red circle for the centroid
        cv2.putText(annotated_image, f"cx: {cx_img}", (cx_img + 10, cy_img), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
    # Draw crop zone and center line on annotated image
    cv2.rectangle(annotated_image, (0, crop_y), (original_w, original_h), (255, 0, 255), 2)
    cv2.line(annotated_image, (center_x_full_image, crop_y), (center_x_full_image, original_h), (0, 255, 255), 2)


    # Convert features_dict to an ordered NumPy array based on FEATURE_COLUMNS
    ordered_features_array = np.array([features_dict[col] for col in FEATURE_COLUMNS]).reshape(1, -1)
    
    return ordered_features_array, annotated_image # Return features and the image with annotations


# --- Main Execution Block ---
if __name__ == "__main__":
    print("--- Starting Lane Deviation Prediction on Unseen Data ---")

    # Load the trained model and scaler
    svr_model, scaler = load_model_and_scaler(MODEL_DIR, SVR_MODEL_FILENAME, SCALER_FILENAME)

    # Compile all tuned parameters into a dictionary for easy passing
    tuned_params = {
        "lower_l": LOWER_L_TUNED, "upper_l": UPPER_L_TUNED, "lower_a": LOWER_A_TUNED, "upper_a": UPPER_A_TUNED,
        "lower_b": LOWER_B_TUNED, "upper_b": UPPER_B_TUNED, "color_morph_kernel_size": COLOR_MORPH_KERNEL_SIZE_TUNED,
        "use_bilateral_filter": USE_BILATERAL_FILTER_TUNED, "bilateral_d": BILATERAL_D_TUNED,
        "bilateral_sigma_color": BILATERAL_SIGMA_COLOR_TUNED, "bilateral_sigma_space": BILATERAL_SIGMA_SPACE_TUNED,
        "canny_thresh1": CANNY_THRESH1_TUNED, "canny_thresh2": CANNY_THRESH2_TUNED,
        "edge_morph_kernel_size": EDGE_MORPH_KERNEL_SIZE_TUNED, "hough_threshold": HOUGH_THRESHOLD_TUNED,
        "hough_min_length": HOUGH_MIN_LENGTH_TUNED, "hough_max_gap": HOUGH_MAX_GAP_TUNED,
        "max_line_angle_deg": MAX_LINE_ANGLE_DEG_TUNED, "crop_percent_from_top": CROP_PERCENT_FROM_TOP_TUNED,
    }

    # Get list of unseen images
    unseen_image_files = [f for f in os.listdir(UNSEEN_DATA_FOLDER) if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))]
    if not unseen_image_files:
        print(f"ERROR: No images found in '{UNSEEN_DATA_FOLDER}'. Please check the path.")
        exit()

    print(f"Found {len(unseen_image_files)} unseen images.")
    print("Press 'q' to quit, 'd' or Right Arrow for next image, 'a' or Left Arrow for previous.")

    current_image_index = 0
    cv2.namedWindow("Prediction Output", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Prediction Output", 1280, 720) # Adjust as needed

    while True:
        filename = unseen_image_files[current_image_index]
        image_path = os.path.join(UNSEEN_DATA_FOLDER, filename)

        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"❌ ERROR: Cannot read {filename}. Skipping to next.")
            current_image_index = (current_image_index + 1) % len(unseen_image_files)
            continue

        # 1. Extract features and get annotated image
        raw_features_array, annotated_display_image = extract_features_for_prediction(original_image, tuned_params)
        
        # Get the 'is_line_detected' flag from the extracted features (it's the first element)
        is_line_detected_flag = raw_features_array[0, FEATURE_COLUMNS.index('is_line_detected')]

        predicted_deviation = float('nan') # Default
        display_text = ""

        if is_line_detected_flag == 1:
            # 2. Scale the extracted features
            scaled_features = scaler.transform(raw_features_array)

            # 3. Predict deviation score
            predicted_deviation = svr_model.predict(scaled_features)[0] # .predict returns an array, get the first element
            display_text = f"Predicted Deviation: {predicted_deviation:.4f}"
        else:
            display_text = "NO LINE DETECTED (No Prediction)"
            predicted_deviation = 0.0 # Assign a neutral value if no line, for consistency if logged

        print(f"\nProcessing {filename}: {display_text}")

        # Add prediction text to the image
        cv2.putText(annotated_display_image, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        
        cv2.imshow("Prediction Output", annotated_display_image)

        # --- Keyboard Controls ---
        key = cv2.waitKey(0) & 0xFF # Wait indefinitely for a key press
        if key == ord("q"): # Press 'q' to quit
            break
        elif key in [ord("d"), 83]: # Press 'd' or Right Arrow to go to the next image
            current_image_index = (current_image_index + 1) % len(unseen_image_files)
        elif key in [ord("a"), 81]: # Press 'a' or Left Arrow to go to the previous image
            current_image_index = (current_image_index - 1 + len(unseen_image_files)) % len(unseen_image_files)

    cv2.destroyAllWindows()
    print("\n--- Prediction session ended. ---")

--- Starting Lane Deviation Prediction on Unseen Data ---
✅ Successfully loaded model from: trained_model\svr_deviation_model.joblib
✅ Successfully loaded scaler from: trained_model\feature_scaler.joblib
Found 1291 unseen images.
Press 'q' to quit, 'd' or Right Arrow for next image, 'a' or Left Arrow for previous.

Processing 00420478.png: NO LINE DETECTED (No Prediction)

Processing 00420523.png: NO LINE DETECTED (No Prediction)

Processing 00420564.png: NO LINE DETECTED (No Prediction)

Processing 00420604.png: NO LINE DETECTED (No Prediction)

Processing 00420644.png: NO LINE DETECTED (No Prediction)

Processing 00420684.png: NO LINE DETECTED (No Prediction)

Processing 00420725.png: NO LINE DETECTED (No Prediction)


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00420765.png: Predicted Deviation: -0.7099

Processing 00420803.png: Predicted Deviation: -0.4208


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00420841.png: Predicted Deviation: -0.2531


...


Processing 00422112.png: Predicted Deviation: -0.4279


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00422150.png: Predicted Deviation: -0.4338


c:\Users\BCI-Lab\.conda\envs\openCV\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Processing 00422187.png: Predicted Deviation: -0.0246
